In [23]:
import spacy
nlp = spacy.load('en')
doc = nlp('Mission Impossible was written by Chrisopher Nolan.')

In [159]:
# statement1 = 'The Mission Impossible was given 7/10 ratings by Rotten Tomatoes.'
# statement1 = 'The Mission Impossible was rated 7/10 ratings by Rotten Tomatoes.'
statement1 = 'Rotten Tomatoes gave 7/10 ratings to the Mission Impossible.'
# statement1 = 'Rotten Tomatoes gave 7/10 ratings to the Mission Impossible.'
# statement1 = '7/10 ratings was given to Mission Impossible by Rotten Tomatoes.'

isValidForRatingTemplate = checkRatingTemplateEligibility(statement1)
if(isValidForRatingTemplate):
    
    movieName = extractMovieName(statement1)
    print("extracted movie name is:" + str(movieName))
    
    ratings = extractRatings(statement1)
    print("extracted Ratings : " , ratings)
    
    ratingsbyWhom = extractGiverName(statement1)
    print("extracted rated by entity is:" , ratingsbyWhom)

Matching with wriiten template: for Rotten Tomatoes gave 7/10 ratings to the Mission Impossible.
Has valid verb:True
vt:gave
to:
child: Impossible
extracted movie name is:the Mission Impossible
vt:gave
extracted Ratings :  7/10
vt:gave
Agent (nsubj) : Rotten Tomatoes
extracted rated by entity is: Rotten Tomatoes


In [83]:
def extractMovieName(statement):
    targetVerbLemmas = ['rat' , 'give'] #rat is a lemma for verb rate
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    movieName = "<uninitialized>"
    
#     handle when release is with premiered.
    '''
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to : "+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    '''

    print("vt:"+verbText)

    movieName = getMovie(statement,verbText)
    
    return movieName

In [88]:
def extractGiverName(statement):
    targetVerbLemmas = ['rat' , 'give']
    
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    writerName = "<uninitialized>"
    
    '''
#     handle when release is with premiered.
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to"+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    '''

    print("vt:"+verbText)

    writerName = getWriter(statement,verbText)
    
    return writerName

In [103]:
def extractRatings(statement):
    targetVerbLemmas = ['rat' , 'give']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    ratings = "<uninitialized>"
    
    print("vt:"+verbText)

    ratings = getRatings(statement,verbText)
    
    return ratings

In [53]:
def getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas):
    """
    this will reutrn exact apperance of verb in statement.
    ex: statement = "Movie was written by James.", targetVerbLemmas = ['write','script']
    this will return "written" as it's lemma matches with one of the targetVerbLemmas
    """
    verbTokens = filterVerbTokens(statement)
    for token in verbTokens:
        if token.lemma_ in targetVerbLemmas:
            return token.text
    return "<no-match>"

In [54]:
def getNounSubject(statement, verbText):
    doc = nlp(statement)
    for token in doc:
        print(token.text, token.dep_, token.head.text)
        if((token.dep_ == "nsubjpass" or token.dep_ == "nsubj") and token.head.text == verbText):
            return token.text
    return "<no-match>"

In [157]:
def getMovie(statement, verbText):
    doc = nlp(statement)
    theme = "<uninitialized>" #movie
    
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "nsubjpass" and token.head.text == verbText):
            print("token nsubjpass : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif (token.text == "to" and token.head.text == verbText):
            print("to:")
            childToken = [child for child in token.children][0]
            print("child:",childToken)
            theme = getNounChunkThatContainsNoun(statement, childToken)
            return theme
        #elif (token.dep_ == "dobj" and token.head.text == verbText):
            #print("token dobj")
            #theme = getNounChunkThatContainsNoun(statement, token)
            #return theme
        elif ((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            print("token conj : " + token.text)
            if (token.head.dep_ == "ROOT"):
                print("token.head: " + token.head.text)
                token1 = token.head
                return getMovie(statement, token1.text)
                '''
                if (token.dep_ == "nsubjpass" and token.head.text == token1.text):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                '''
    
    return "<no-match>"

In [137]:
def getWriter(statement, verbText):
    doc = nlp(statement)
    agent = "<uninitialized>" #writer
    '''
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        elif (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
    '''
    
    for token in doc:
        #print("token:",token)
        '''
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            print("Agent (agent) : " + agent)
            return agent
        '''
        if (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            print("Agent (nsubj) : " + agent)
            return agent
        
        elif((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        
        elif(token.head.text == verbText or token.text == verbText):
            print(token.head.text)
            agent = extractOwnerEntitySyntactically(statement)
            return agent
        
    return agent

In [115]:
def extractOwnerEntitySyntactically(statement):
    doc = nlp(statement)
    verbTokens = filterVerbTokens(statement)
    ORGEntity = []
    
    for token in verbTokens:
        if isExpectedVerbToken(token):
            #print("Expected token is:"+token.text)
            #expected children [was, theatrically, in, on, by]
            byToken = None
            for child in token.children: 
                #usually release location starts with "in"
                if child.text == "by":
                    byToken = child
             
            if byToken == None:
                #means no direct by available so lets look at some conj child
                for child in token.children: 
                    #usually release location starts with "in"
                    if child.dep_ == "conj":
                        for grandChild in child.children:
                            if grandChild.text == "by":
                                byToken = grandChild
                    else:
                        for grandChild in child.children:
                            if grandChild.text == "by":
                                byToken = grandChild
            
            print("byToken:" , byToken)
            
            if byToken != None:
                for subChild in byToken.children:
                    ORGEntity.append(getORGorPersonThatContainsToken(statement, subChild))
                    print("subchild : " + subChild.text)
                    if (subChild.dep_ == "pobj" and subChild.head.text == "by"):
                        grandChildren = subChild.children
                        childEntity = getORGEntityForGrandChild1(statement, grandChildren, ORGEntity)
                        #print("child entity:    " , str(childEntity))
                        return list(set(childEntity))
                        #ORGEntity.clear()
                        #for item in childEntity:
                            #ORGEntity.append(item)
                    return ORGEntity
        else: 
            print(isExpectedVerbToken(token))
    return "<no-match>"

In [58]:
def getORGEntityForGrandChild1(statement, children, ORGEntity):
    '''
    New definition
    '''
    doc = nlp(statement)
    
    for child in children:
        sizeChild = sum(1 for item in child.children)
        print(child, sizeChild)
        if sizeChild > 0:
            ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
            print(ORGEntity)
            #ORGEntity.append(getORGEntityForGrandChild1(statement, child.children, ORGEntity))
            getORGEntityForGrandChild1(statement, child.children, ORGEntity)
        if sizeChild == 0:
            if (getORGorPersonThatContainsToken(statement, child) != "<no-match>"):
                ORGEntity.append(getORGorPersonThatContainsToken(statement, child))
    
    return ORGEntity

In [123]:
def getRatings(statement, verbText):
    doc = nlp(statement)
    
    ratings = "<uninitialized>"
    
    if (check_word_pattern(statement, " ratings ")):
        for token in doc:
            if (token.head.text == "ratings"):
                ratings = getORGorPersonThatContainsToken(statement, token)
                return ratings
        
    else:
        for token in doc:
            if (token.head.text == verbText and (token.dep_ == "dobj" or token.dep_ == "oprd")):
                ratings = getORGorPersonThatContainsToken(statement, token)
                return ratings

In [96]:
def isExpectedVerbToken(token):
    return (token.lemma_ in ["give"] or token.lemma_ in ["rat"])

In [60]:
def getORGorPersonThatContainsToken(statement, token):
    doc = nlp (statement)
    tokenStartPos = calculateStartPositionOfToken(statement, token)
    for ent in doc.ents:
        if(token.text in ent.text and tokenStartPos >= ent.start_char and tokenStartPos <= ent.end_char):
            return ent.text
    return "<no-match>"

In [61]:
import re
def calculateStartPositionOfToken(statement, targetToken):
    """
    this should return start position of the token
    """
    matches = [m.start() for m in re.finditer(targetToken.text, statement)]
    if(len(matches)==1):
        return matches[0]
    

    huristicPosition = 0
    doc = nlp(statement)
    
    i = 0
    for token in doc:
        if i < targetToken.i:
            huristicPosition = huristicPosition + len(token.text)+1
            i = i+1
        else:
            break
        
    distances = []
    for match in matches:
        distances.append(abs(match-huristicPosition))
        
    minIndex = distances.index(min(distances))
    
    return matches[minIndex]

In [62]:
def getNounChunkThatContainsNoun(statement, nounSubject):
    doc = nlp(statement)
    for chunk in doc.noun_chunks:
        if nounSubject.text in chunk.text:
            return chunk.text
    return "<no-match>"

In [63]:
def checkRatingTemplateEligibility(statement):
    print("Matching with wriiten template: for " + statement)
    hasValidVerb = check_verb_match(statement)
    print("Has valid verb:" + str(hasValidVerb))
    
    return hasValidVerb

In [84]:
def check_verb_match(statement):
    verbTokens = filterVerbTokens(statement)
    
    targetVerbs = ['rat', 'give']
    for token in verbTokens:
        if token.lemma_ in targetVerbs:
            return True
    return False

In [106]:
def check_word_pattern(statement, pattern1):
    return ((pattern1 in statement))

In [85]:
check_verb_match('The Mission Impossible was rated 7/10 by Rotten Tomatoes.')


True

In [69]:
def filterVerbTokens(statement):
    doc = nlp(statement)
    result = list(filter(lambda token: token.pos_ == "VERB" and token.lemma_ != "be", doc))
    return result

In [70]:
filterVerbTokens('The Mission Impossible was rated 7/10 by Rotten Tomatoes.')

[rated]

In [66]:
def get_dependency_tree_nodes(sentence):
    nodes = []
    doc = nlp(sentence)
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [122]:
get_dependency_tree_nodes('The Mission Impossible was rated 7/10 by Rotten Tomatoes.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'rated', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'rated', 'VERB', []],
 ['rated', 'ROOT', 'rated', 'VERB', [Impossible, was, 7/10, by, .]],
 ['7/10', 'oprd', 'rated', 'VERB', []],
 ['by', 'agent', 'rated', 'VERB', [Tomatoes]],
 ['Rotten', 'compound', 'Tomatoes', 'PROPN', []],
 ['Tomatoes', 'pobj', 'by', 'ADP', [Rotten]],
 ['.', 'punct', 'rated', 'VERB', []]]

In [101]:
get_dependency_tree_nodes('The Mission Impossible was given 7/10 ratings by Rotten Tomatoes.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'given', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'given', 'VERB', []],
 ['given', 'ROOT', 'given', 'VERB', [Impossible, was, ratings, .]],
 ['7/10', 'nummod', 'ratings', 'NOUN', []],
 ['ratings', 'dobj', 'given', 'VERB', [7/10, by]],
 ['by', 'prep', 'ratings', 'NOUN', [Tomatoes]],
 ['Rotten', 'compound', 'Tomatoes', 'PROPN', []],
 ['Tomatoes', 'pobj', 'by', 'ADP', [Rotten]],
 ['.', 'punct', 'given', 'VERB', []]]

In [102]:
get_dependency_tree_nodes('7/10 ratings was given to Mission Impossible by Rotten Tomatoes.')

[['7/10', 'nummod', 'ratings', 'NOUN', []],
 ['ratings', 'nsubjpass', 'given', 'VERB', [7/10]],
 ['was', 'auxpass', 'given', 'VERB', []],
 ['given', 'ROOT', 'given', 'VERB', [ratings, was, to, by, .]],
 ['to', 'dative', 'given', 'VERB', [Impossible]],
 ['Mission', 'compound', 'Impossible', 'ADJ', []],
 ['Impossible', 'pobj', 'to', 'ADP', [Mission]],
 ['by', 'agent', 'given', 'VERB', [Tomatoes]],
 ['Rotten', 'compound', 'Tomatoes', 'PROPN', []],
 ['Tomatoes', 'pobj', 'by', 'ADP', [Rotten]],
 ['.', 'punct', 'given', 'VERB', []]]

In [142]:
get_dependency_tree_nodes('Rotten Tomatoes gave 7/10 ratings to the Mission Impossible.')

[['Rotten', 'compound', 'Tomatoes', 'NOUN', []],
 ['Tomatoes', 'nsubj', 'gave', 'VERB', [Rotten]],
 ['gave', 'ROOT', 'gave', 'VERB', [Tomatoes, ratings, to, .]],
 ['7/10', 'nummod', 'ratings', 'NOUN', []],
 ['ratings', 'dobj', 'gave', 'VERB', [7/10]],
 ['to', 'dative', 'gave', 'VERB', [Impossible]],
 ['the', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'pobj', 'to', 'ADP', [the, Mission]],
 ['.', 'punct', 'gave', 'VERB', []]]

In [120]:
doc = nlp ("The Mission Impossible was rated 7/10 by Rotten Tomatoes.")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

7/10 33 37 CARDINAL
Rotten Tomatoes 41 56 PERSON
